These Job Summaries were initially developed to filter the text to the most important information. Unfortunately, the model proved to be ineffective, and as a result, the summaries were not utilized.

In [ ]:
import numpy as np
import pandas as pd

import torch
from transformers import BartForConditionalGeneration, BartTokenizer
import textwrap


In [ ]:
df = pd.read_csv('dfCleaned.csv', engine='python')

In [ ]:
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

def textSummary(text):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True, ) #padding="max_length")
    summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    formatted_summary = " ".join(textwrap.wrap(summary, width=80))
    return formatted_summary

df['description summary'] = pd.Series(dtype="string")

In [ ]:
# Ideally, this line would be used to generate the summaries.  Due to system timeouts, the task had to be divided.

#df['description summary'] = df['descriptionShort'].apply(lambda text: textSummary(text))

In [ ]:
# This line was utilized repeatedly to divide the task. On my systems, processing every hundred records requires 10 minutes.

def splitSummary(lower, upper):
    df['description summary'][lower:upper] = df['descriptionShort'][lower:upper].apply(lambda text: textSummary(text))

In [ ]:
splitSummary(450, 507)

df[df['description summary'].isnull()]

In [ ]:
df.to_csv('dfSummary.csv', index=False)